In [2]:
import torch
from torch import nn

# ModuleList vs Sequential

In [5]:
fc=nn.Linear(3,3)
layer_list = [fc for _ in range(5)]

layers1 = nn.Sequential(*layer_list)
print(layers1)
print("-"*50)
layers2 = nn.ModuleList(layer_list) # 동일한 역할, 하지만 얘는 리스트로 넘겨줘도 된다
print(layers2)

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=3, bias=True)
  (2): Linear(in_features=3, out_features=3, bias=True)
  (3): Linear(in_features=3, out_features=3, bias=True)
  (4): Linear(in_features=3, out_features=3, bias=True)
)
--------------------------------------------------
ModuleList(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=3, bias=True)
  (2): Linear(in_features=3, out_features=3, bias=True)
  (3): Linear(in_features=3, out_features=3, bias=True)
  (4): Linear(in_features=3, out_features=3, bias=True)
)


In [6]:
x=torch.randn(1,3)
print(layers1(x))
print(layers2(x))

tensor([[ 0.9248, -0.6928,  1.0017]], grad_fn=<AddmmBackward0>)


NotImplementedError: Module [ModuleList] is missing the required "forward" function

nn.ModuleList는 forward가 내장되어 있지 않다

In [7]:
# 리스트처럼 하나씩 다뤄서 통과해야 가능하다
for layer in layers2:
    x = layer(x)
print(x)

tensor([[ 0.9248, -0.6928,  1.0017]], grad_fn=<AddmmBackward0>)


In [8]:
layer_list = [fc for _ in range(5)]
for layer in layer_list:
    x = layer(x)
print(x)

tensor([[ 1.1701, -0.3753,  0.8959]], grad_fn=<AddmmBackward0>)


근데 리스트도 동일한 역할인데...

In [11]:
class testNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.Module_List = [nn.Linear(3,3), nn.Linear(3,3)]
        # self.Module_List = nn.ModuleList([nn.Linear(3,3), nn.Linear(3,3)])

    def forward(self,x):
        for layer in self.Module_List:
            x = layer(x)
        return x

model=testNet()
print(model(torch.randn(1,3)))
print("-"*50)
# 그냥 리스트로 하면 등록이 안돼있다!
print(model)
list(model.parameters())

tensor([[0.0171, 0.2603, 0.3912]], grad_fn=<AddmmBackward0>)
--------------------------------------------------
testNet()


[]

In [12]:
# 그럼 nn.Sequential 쓰고 말지 왜 굳이 nn.ModuleList?
class small_block(nn.Module):
    def __init__(self):
        super().__init__()
        self.block_x = nn.Linear(1,1)
        self.block_y = nn.Linear(1,1)

    def forward(self, x, y):
        x = self.block_x(x)
        y = self.block_y(y)
        return x, y

In [17]:
block = small_block()
model = nn.Sequential(block)
print(model)
model(torch.randn(1), torch.randn(1)) # nn.Sequential 이 가지고 있는 forward 함수 > 입력 하나만 받음

Sequential(
  (0): small_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
)


TypeError: forward() takes 2 positional arguments but 3 were given

In [19]:
model = nn.ModuleList([block])
print(model)
print("-"*50)
x = torch.randn(1)
y = torch.randn(1)
for block in model:
    x, y = block(x,y)
print(x, y)

ModuleList(
  (0): small_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
)
--------------------------------------------------
tensor([0.8309], grad_fn=<AddBackward0>) tensor([-0.6931], grad_fn=<AddBackward0>)
